In [ ]:
from openai import OpenAI
import os
import pandas as pd
from sklearn.metrics import classification_report

# Configurar API
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Exemplos para few-shot learning
exemplos = """
Positivas:
"Terá todo o dinheiro do mundo, mas há algo que nunca poderá comprar ... um dinossauro!"
"Olha para mim! Estou a fazer as pessoas felizes, sou o homem mágico, do país feliz, da casa da geleia, da rua do pirulito."
"Muito bom, ainda serve."
"É engraçado porque é verdade"
"Adoro quando isso acontece."

Negativas:
"Crianças, vocês tentaram muito. Para quê? Para fazer de si mesmo um tolo. A moral é: não se esforce."
"Sem TV e sem cerveja, Homer perde a cabeça."
"Se não gosta do seu trabalho, não faça greve, continue todos os dias e continue a fazê-lo a meio caminho. É assim que é americano!"
"Não me coma, tenho mulher e filhos, coma-os."
"Não diga vingança ... não diga vingança ... Vingança!"

Neutras:
"E se cometermos um erro sobre religião? Deus ficaria mais furioso todas as semanas."
"Podes dizer-me onde está a pia? Gostaria de fingir que lavo as mãos."
"Salada não leva a nada."
"Queria que Deus estivesse vivo para ver isso."
"Estou lendo um livro interessante."
"""

# Carregar dataset
dataset = pd.read_csv("../data/simpson_show_df.csv")
episodio_92_temporada_5 = dataset[(dataset['episode_id'] == 92) & (dataset['episode_season'] == 5)]
falas_episodio = episodio_92_temporada_5['spoken_words'].tolist()

# Dividir falas em lotes 
batch_size = 10
lotes = [falas_episodio[i:i + batch_size] for i in range(0, len(falas_episodio), batch_size)]

# Normalizar rótulos de saída
def normalizar_rotulo(rotulo):
    rotulo = rotulo.strip().lower()
    if "positiva" in rotulo:
        return "Positiva"
    elif "negativa" in rotulo:
        return "Negativa"
    elif "neutra" in rotulo:
        return "Neutra"
    else:
        return "Neutra"  

# few-shot learning
resultados = []
for lote in lotes:
    falas_texto = "\n".join([f"Fala: \"{fala}\"" for fala in lote])
    prompt = f"""
    Você é um especialista em análise de sentimentos. Baseie-se nos exemplos a seguir:
    {exemplos}
    Classifique as falas abaixo como Positivas, Neutras ou Negativas:
    {falas_texto}
    """
    
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=150,
        temperature=1,
    )
    
    classificacoes = [normalizar_rotulo(c.strip()) for c in response.choices[0].text.strip().split('\n')]
    resultados.extend(classificacoes)

# Quantidade de chamadas ao LLM
chamadas_llm = len(lotes)

distribuicao = {"Positivas": 0, "Neutras": 0, "Negativas": 0}
for resultado in resultados:
    if resultado not in distribuicao:
        distribuicao[resultado] = 0 
    distribuicao[resultado] += 1


# Avaliar o modelo com subconjunto de validação
falas_validacao = [
    "Isso é ótimo!", "Horrível, não suporto mais isso.", "Apenas mais um dia comum.",
    "Que incrível resultado!", "Essa foi péssima.", "Nada de especial aconteceu hoje."
]
labels_verdadeiros = ["Positiva", "Negativa", "Neutra", "Positiva", "Negativa", "Neutra"]

# Criar prompt para validação
falas_validacao_prompt = "\n".join([f"Fala: \"{fala}\"" for fala in falas_validacao])
prompt_validacao = f"""
Você é um especialista em análise de sentimentos. Baseie-se nos exemplos a seguir:
{exemplos}
Classifique as falas abaixo como Positivas, Neutras ou Negativas:
{falas_validacao_prompt}
"""             

response_validacao = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_validacao,
    max_tokens=150,
    temperature=1,
)

all_labels = set(labels_verdadeiros + [normalizar_rotulo(res.strip()) for res in response_validacao.choices[0].text.strip().split('\n')])

# Avaliação do modelo
relatorio = classification_report(
    labels_verdadeiros,
    [normalizar_rotulo(res.strip()) for res in response_validacao.choices[0].text.strip().split('\n')],
    target_names=list(all_labels),  
    zero_division=0 
)

# Resultados
print("Quantas chamadas ao LLM foram necessárias:", chamadas_llm)
print("Distribuição de fala por categoria:", distribuicao)
print("Relatório de avaliação:\n", relatorio)


C:\Users\gabri\AppData\Local\Temp\ipykernel_5768\600062901.py:34: DtypeWarning: Columns (4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("../src/simpson_show_df.csv")


Quantas chamadas ao LLM foram necessárias: 28
Distribuição de fala por categoria: {'Positivas': 0, 'Neutras': 0, 'Negativas': 0, 'Negativa': 51, 'Neutra': 124, 'Positiva': 48}
Relatório de avaliação:
               precision    recall  f1-score   support

    Negativa       1.00      1.00      1.00         2
      Neutra       1.00      1.00      1.00         2
    Positiva       1.00      1.00      1.00         2

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



In [ ]:
# Carregar dataset
dataset = pd.read_csv("../data/simpson_show_df.csv")
episodio_92_temporada_5 = dataset[(dataset['episode_id'] == 92) & (dataset['episode_season'] == 5)]
falas_episodio = episodio_92_temporada_5['spoken_words'].tolist()


C:\Users\gabri\AppData\Local\Temp\ipykernel_5768\4055441034.py:2: DtypeWarning: Columns (4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("../src/simpson_show_df.csv")


In [49]:
# Ajustar tamanhos para serem iguais
min_tamanho = min(len(falas_episodio), len(resultados))

falas_episodioo = falas_episodio[:min_tamanho]
resultadoss = resultados[:min_tamanho]

df_resultados = pd.DataFrame({
    "Fala": falas_episodioo,
    "Categoria": resultadoss
})

df_resultados.dropna(inplace=True)


In [62]:


df_resultados.to_csv("ep92se5_classification.csv", index=False)

In [55]:
print("Tamanho de falas_episodio:", len(falas_episodio))
print("Tamanho de resultados:", len(resultados))


Tamanho de falas_episodio: 277
Tamanho de resultados: 223


In [ ]:
for i, lote in enumerate(lotes):
    print(f"Lote {i + 1}: {len(lote)} falas"
    print(f"Classificações retornadas para o lote: {len(resultados[i * batch_size: (i + 1) * batch_size])}")


Lote 1: 10 falas
Classificações retornadas para o lote: 10
Lote 2: 10 falas
Classificações retornadas para o lote: 10
Lote 3: 10 falas
Classificações retornadas para o lote: 10
Lote 4: 10 falas
Classificações retornadas para o lote: 10
Lote 5: 10 falas
Classificações retornadas para o lote: 10
Lote 6: 10 falas
Classificações retornadas para o lote: 10
Lote 7: 10 falas
Classificações retornadas para o lote: 10
Lote 8: 10 falas
Classificações retornadas para o lote: 10
Lote 9: 10 falas
Classificações retornadas para o lote: 10
Lote 10: 10 falas
Classificações retornadas para o lote: 10
Lote 11: 10 falas
Classificações retornadas para o lote: 10
Lote 12: 10 falas
Classificações retornadas para o lote: 10
Lote 13: 10 falas
Classificações retornadas para o lote: 10
Lote 14: 10 falas
Classificações retornadas para o lote: 10
Lote 15: 10 falas
Classificações retornadas para o lote: 10
Lote 16: 10 falas
Classificações retornadas para o lote: 10
Lote 17: 10 falas
Classificações retornadas para 